# 챕터 3: OpenAI API Endpoint와 비교하여 구현

## 3-1. Claude API vs OpenAI API 개요

두 API의 주요 차이점:
- **메시지 구조**: 둘 다 messages 배열을 사용하지만 세부 구조가 다르다
- **System Prompt**: Claude는 system 파라미터, OpenAI는 messages 내 system role
- **응답 형식**: 응답 구조가 다르다
- **모델 이름**: 각자 다른 모델명을 사용한다

이번 챕터에서는 두 API를 모두 지원하는 통합 Skills 시스템을 구축한다.

In [ ]:
# 필요한 라이브러리 설치
%pip install -q anthropic openai python-dotenv pyyaml

In [1]:
import anthropic
import openai
import os
from pathlib import Path
import yaml
from typing import Dict, List, Optional, Union, Literal
from dataclasses import dataclass
from abc import ABC, abstractmethod
from enum import Enum

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## 3-2. API 제공자 열거형 정의

지원하는 AI 제공자를 명확히 정의한다.

In [22]:
CLAUDE_MODEL = "claude-sonnet-4-5"
OPENAI_MODEL = "gpt-4o-mini"

In [23]:
class AIProvider(Enum):
    """
    지원하는 AI API 제공자
    """
    ANTHROPIC = "anthropic"
    OPENAI = "openai"

@dataclass
class ModelConfig:
    """
    각 제공자별 모델 설정
    """
    provider: AIProvider
    model_name: str
    max_tokens: int = 2000
    temperature: float = 0.7

# 각 제공자별 기본 모델 설정
DEFAULT_MODELS = {
    AIProvider.ANTHROPIC: ModelConfig(
        provider=AIProvider.ANTHROPIC,
        model_name=CLAUDE_MODEL,
        max_tokens=2000
    ),
    AIProvider.OPENAI: ModelConfig(
        provider=AIProvider.OPENAI,
        model_name=OPENAI_MODEL,
        max_tokens=2000
    )
}

print("모델 설정 완료")
print(f"  - Anthropic 기본 모델: {DEFAULT_MODELS[AIProvider.ANTHROPIC].model_name}")
print(f"  - OpenAI 기본 모델: {DEFAULT_MODELS[AIProvider.OPENAI].model_name}")

모델 설정 완료
  - Anthropic 기본 모델: claude-sonnet-4-5
  - OpenAI 기본 모델: gpt-4o-mini


## 3-3. 추상 베이스 클래스로 AI Client 인터페이스 정의

다양한 AI 제공자를 지원하기 위한 공통 인터페이스를 정의한다.

In [24]:
class AIClient(ABC):
    """
    AI API 클라이언트의 추상 베이스 클래스
    모든 AI 제공자는 이 인터페이스를 구현해야 한다
    """
    
    @abstractmethod
    def __init__(self, api_key: Optional[str] = None):
        """
        클라이언트를 초기화한다
        
        Args:
            api_key: API 키 (None이면 환경 변수에서 가져온다)
        """
        pass
    
    @abstractmethod
    def create_completion(
        self,
        system_prompt: str,
        user_message: str,
        model: str,
        max_tokens: int,
        temperature: float
    ) -> str:
        """
        AI 모델에게 completion을 요청한다
        
        Args:
            system_prompt: 시스템 프롬프트 (Skill 내용)
            user_message: 사용자 메시지
            model: 사용할 모델 이름
            max_tokens: 최대 토큰 수
            temperature: 온도 파라미터
            
        Returns:
            AI의 응답 텍스트
        """
        pass
    
    @abstractmethod
    def get_provider(self) -> AIProvider:
        """
        현재 클라이언트의 제공자를 반환한다
        
        Returns:
            AIProvider 열거형
        """
        pass

print("AIClient 추상 클래스 정의 완료")

AIClient 추상 클래스 정의 완료


## 3-4. Anthropic Client 구현

Claude API를 사용하는 클라이언트를 구현한다.

In [25]:
class AnthropicClient(AIClient):
    """
    Anthropic Claude API 클라이언트
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Anthropic 클라이언트를 초기화한다
        
        Args:
            api_key: Anthropic API 키
        """
        self.client = anthropic.Anthropic(
            api_key=os.environ.get("ANTHROPIC_API_KEY")
        )
    
    def create_completion(
        self,
        system_prompt: str,
        user_message: str,
        model: str,
        max_tokens: int,
        temperature: float = 0.7
    ) -> str:
        """
        Claude API를 호출하여 completion을 생성한다
        
        Claude API 특징:
        - system 파라미터로 시스템 프롬프트를 전달한다
        - messages 배열에는 user/assistant role만 포함한다
        - response.content[0].text로 응답을 추출한다
        """
        response = self.client.messages.create(
            model=model,
            max_tokens=max_tokens,
            temperature=temperature,
            system=system_prompt,  # Claude는 system을 별도 파라미터로 받는다
            messages=[
                {
                    "role": "user",
                    "content": user_message
                }
            ]
        )
        
        return response.content[0].text
    
    def get_provider(self) -> AIProvider:
        return AIProvider.ANTHROPIC

print("AnthropicClient 구현 완료")

AnthropicClient 구현 완료


## 3-5. OpenAI Client 구현

OpenAI API를 사용하는 클라이언트를 구현한다.

In [26]:
class OpenAIClient(AIClient):
    """
    OpenAI API 클라이언트
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        OpenAI 클라이언트를 초기화한다
        
        Args:
            api_key: OpenAI API 키
        """
        self.client = openai.OpenAI(
            api_key=os.environ.get("OPENAI_API_KEY")
        )
    
    def create_completion(
        self,
        system_prompt: str,
        user_message: str,
        model: str,
        max_tokens: int,
        temperature: float = 0.7
    ) -> str:
        """
        OpenAI API를 호출하여 completion을 생성한다
        
        OpenAI API 특징:
        - messages 배열 내에 system role을 포함한다
        - system 메시지를 첫 번째로 배치한다
        - response.choices[0].message.content로 응답을 추출한다
        """
        response = self.client.chat.completions.create(
            model=model,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[
                {
                    "role": "system",  # OpenAI는 messages 배열 안에 system을 넣는다
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_message
                }
            ]
        )
        
        return response.choices[0].message.content
    
    def get_provider(self) -> AIProvider:
        return AIProvider.OPENAI

print("OpenAIClient 구현 완료")

OpenAIClient 구현 완료


## 3-6. 통합 Skill Agent 구현

두 API 제공자를 모두 지원하는 통합 에이전트를 구현한다.

In [27]:
class UnifiedSkillAgent:
    """
    여러 AI 제공자를 지원하는 통합 Skill Agent
    """
    
    def __init__(
        self,
        provider: AIProvider = AIProvider.ANTHROPIC,
        anthropic_api_key: Optional[str] = None,
        openai_api_key: Optional[str] = None
    ):
        """
        통합 에이전트를 초기화한다
        
        Args:
            provider: 기본으로 사용할 AI 제공자
            anthropic_api_key: Anthropic API 키
            openai_api_key: OpenAI API 키
        """
        self.provider = provider
        self.clients: Dict[AIProvider, AIClient] = {}
        
        # Anthropic 클라이언트 초기화
        try:
            self.clients[AIProvider.ANTHROPIC] = AnthropicClient(anthropic_api_key)
            print(f"Anthropic 클라이언트 초기화 완료")
        except Exception as e:
            print(f"Anthropic 클라이언트 초기화 실패: {e}")
        
        # OpenAI 클라이언트 초기화
        try:
            self.clients[AIProvider.OPENAI] = OpenAIClient(openai_api_key)
            print(f"OpenAI 클라이언트 초기화 완료")
        except Exception as e:
            print(f"OpenAI 클라이언트 초기화 실패: {e}")
        
        # Skill 저장소
        self.skills: Dict[str, Dict] = {}
    
    def set_provider(self, provider: AIProvider) -> None:
        """
        기본 AI 제공자를 변경한다
        
        Args:
            provider: 새로운 기본 제공자
        """
        if provider not in self.clients:
            raise ValueError(f"{provider.value} 클라이언트가 초기화되지 않았다")
        self.provider = provider
        print(f"기본 제공자를 {provider.value}로 변경했다")
    
    def load_skill_from_file(self, skill_path: str) -> Dict:
        """
        파일에서 Skill을 로드한다
        
        Args:
            skill_path: Skill 파일 경로 (.yaml 또는 .md)
            
        Returns:
            로드된 Skill 데이터
        """
        path = Path(skill_path)
        
        if path.suffix == '.yaml':
            with open(path, 'r', encoding='utf-8') as f:
                skill_data = yaml.safe_load(f)
        elif path.suffix == '.md':
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
            skill_data = {
                'name': path.stem,
                'content': content,
                'version': '1.0.0'
            }
        else:
            raise ValueError(f"지원하지 않는 파일 형식이다: {path.suffix}")
        
        skill_name = skill_data['name']
        self.skills[skill_name] = skill_data
        print(f"Skill '{skill_name}' 로드 완료")
        
        return skill_data
    
    def execute(
        self,
        skill_name: str,
        user_message: str,
        provider: Optional[AIProvider] = None,
        model: Optional[str] = None,
        max_tokens: Optional[int] = None,
        temperature: Optional[float] = None
    ) -> Dict[str, Union[str, AIProvider]]:
        """
        Skill을 실행하여 AI 응답을 생성한다
        
        Args:
            skill_name: 실행할 Skill 이름
            user_message: 사용자 메시지
            provider: 사용할 AI 제공자 (None이면 기본값)
            model: 사용할 모델 (None이면 기본값)
            max_tokens: 최대 토큰 수
            temperature: 온도 파라미터
            
        Returns:
            응답 딕셔너리 (response, provider, model 포함)
        """
        # Skill 확인
        if skill_name not in self.skills:
            raise ValueError(f"Skill '{skill_name}'을 찾을 수 없다")
        
        skill_data = self.skills[skill_name]
        
        # 제공자 결정
        selected_provider = provider or self.provider
        if selected_provider not in self.clients:
            raise ValueError(f"{selected_provider.value} 클라이언트를 사용할 수 없다")
        
        client = self.clients[selected_provider]
        
        # 모델 설정 결정 (우선순위: 파라미터 > Skill 설정 > 기본값)
        skill_config = skill_data.get('config', {})
        default_config = DEFAULT_MODELS[selected_provider]
        
        final_model = model or skill_config.get('model') or default_config.model_name
        final_max_tokens = max_tokens or skill_config.get('max_tokens') or default_config.max_tokens
        final_temperature = temperature if temperature is not None else skill_config.get('temperature', default_config.temperature)
        
        # AI API 호출
        response_text = client.create_completion(
            system_prompt=skill_data['content'],
            user_message=user_message,
            model=final_model,
            max_tokens=final_max_tokens,
            temperature=final_temperature
        )
        
        return {
            'response': response_text,
            'provider': selected_provider,
            'model': final_model
        }
    
    def list_skills(self) -> List[str]:
        """
        로드된 모든 Skill 목록을 반환한다
        
        Returns:
            Skill 이름 리스트
        """
        return list(self.skills.keys())
    
    def list_available_providers(self) -> List[AIProvider]:
        """
        사용 가능한 AI 제공자 목록을 반환한다
        
        Returns:
            AIProvider 리스트
        """
        return list(self.clients.keys())

print("UnifiedSkillAgent 구현 완료")

UnifiedSkillAgent 구현 완료


## 3-7. 테스트용 Skill 생성

In [28]:
# skills 디렉토리 생성
skills_dir = Path("skills/skills_comparison")
skills_dir.mkdir(exist_ok=True)

# 번역가 Skill 생성
translator_skill = """# 전문 번역가 Skill

## 역할
텍스트를 자연스럽고 정확하게 번역하는 전문 번역가다.

## 번역 원칙
1. 정확성: 원문의 의미를 정확히 전달한다
2. 자연스러움: 목표 언어의 자연스러운 표현을 사용한다
3. 문맥 고려: 문맥에 맞는 적절한 단어를 선택한다
4. 문화적 적절성: 문화적 차이를 고려한다

## 출력 형식
- 번역된 텍스트만 출력한다
- 추가 설명이나 주석은 요청이 있을 때만 포함한다

## 스타일
- 문장은 ~다 체를 사용한다
- 전문적이고 정확한 어조를 유지한다
"""

translator_yaml = {
    'name': 'translator',
    'version': '1.0.0',
    'description': '전문 번역 서비스',
    'content': translator_skill,
    'config': {
        'max_tokens': 1500,
        'temperature': 0.3
    }
}

# YAML 파일로 저장
translator_path = skills_dir / 'translator.yaml'
with open(translator_path, 'w', encoding='utf-8') as f:
    yaml.dump(translator_yaml, f, allow_unicode=True, default_flow_style=False)

print(f"번역가 Skill 생성: {translator_path}")

번역가 Skill 생성: skills_comparison/translator.yaml


## 3-8. 통합 에이전트 사용 예제

In [29]:
# 통합 에이전트 생성
agent = UnifiedSkillAgent(provider=AIProvider.ANTHROPIC)

# 사용 가능한 제공자 확인
print("\n사용 가능한 AI 제공자:")
for provider in agent.list_available_providers():
    print(f"  - {provider.value}")

Anthropic 클라이언트 초기화 완료
OpenAI 클라이언트 초기화 완료

사용 가능한 AI 제공자:
  - anthropic
  - openai


In [30]:
# Skill 로드
agent.load_skill_from_file('skills/skills_comparison/translator.yaml')

print("\n로드된 Skills:")
for skill in agent.list_skills():
    print(f"  - {skill}")

Skill 'translator' 로드 완료

로드된 Skills:
  - translator


In [31]:
# Claude로 번역 테스트
print("\n" + "=" * 80)
print("테스트 1: Claude (Anthropic)로 번역")
print("=" * 80)

text_to_translate = """
Artificial Intelligence is transforming the way we work and live.
It has applications in healthcare, finance, education, and many other fields.
"""

result1 = agent.execute(
    skill_name='translator',
    user_message=f"다음 영어 텍스트를 한국어로 번역해줘:\n{text_to_translate}",
    provider=AIProvider.ANTHROPIC
)

print(f"제공자: {result1['provider'].value}")
print(f"모델: {result1['model']}")
print(f"\n번역 결과:\n{result1['response']}")


테스트 1: Claude (Anthropic)로 번역
제공자: anthropic
모델: claude-sonnet-4-5

번역 결과:
인공지능은 우리가 일하고 생활하는 방식을 변화시키고 있다.
인공지능은 의료, 금융, 교육 및 기타 많은 분야에서 응용되고 있다.


In [32]:
# OpenAI로 번역 테스트 (API 키가 설정된 경우)
print("\n" + "=" * 80)
print("테스트 2: GPT (OpenAI)로 번역")
print("=" * 80)

try:
    result2 = agent.execute(
        skill_name='translator',
        user_message=f"다음 영어 텍스트를 한국어로 번역해줘:\n{text_to_translate}",
        provider=AIProvider.OPENAI
    )
    
    print(f"제공자: {result2['provider'].value}")
    print(f"모델: {result2['model']}")
    print(f"\n번역 결과:\n{result2['response']}")
except Exception as e:
    print(f"OpenAI 실행 실패: {e}")
    print("OpenAI API 키가 설정되지 않았거나 유효하지 않다")


테스트 2: GPT (OpenAI)로 번역
제공자: openai
모델: gpt-4o-mini

번역 결과:
인공지능은 우리가 일하고 생활하는 방식을 변화시키고 있다. 그것은 의료, 금융, 교육 및 많은 다른 분야에서 응용되고 있다.


## 3-9. 제공자별 비교 테스트

동일한 Skill과 입력으로 두 제공자의 응답을 비교한다.

In [33]:
def compare_providers(
    agent: UnifiedSkillAgent,
    skill_name: str,
    user_message: str
) -> Dict[str, Dict]:
    """
    여러 제공자로 동일한 작업을 수행하고 결과를 비교한다
    
    Args:
        agent: UnifiedSkillAgent 인스턴스
        skill_name: 사용할 Skill 이름
        user_message: 사용자 메시지
        
    Returns:
        제공자별 결과 딕셔너리
    """
    results = {}
    
    for provider in agent.list_available_providers():
        try:
            print(f"\n{provider.value}로 실행 중...")
            result = agent.execute(
                skill_name=skill_name,
                user_message=user_message,
                provider=provider
            )
            results[provider.value] = result
        except Exception as e:
            print(f"{provider.value} 실행 실패: {e}")
            results[provider.value] = {'error': str(e)}
    
    return results

# 비교 테스트 실행
print("\n" + "=" * 80)
print("제공자별 비교 테스트")
print("=" * 80)

comparison_message = "Hello, how are you today?를 한국어로 번역해줘"
comparison_results = compare_providers(agent, 'translator', comparison_message)

# 결과 출력
for provider_name, result in comparison_results.items():
    print(f"\n[{provider_name.upper()}]")
    if 'error' in result:
        print(f"  에러: {result['error']}")
    else:
        print(f"  모델: {result['model']}")
        print(f"  응답: {result['response']}")


제공자별 비교 테스트

anthropic로 실행 중...

openai로 실행 중...

[ANTHROPIC]
  모델: claude-sonnet-4-5
  응답: 안녕하세요, 오늘 어떻게 지내세요?

[OPENAI]
  모델: gpt-4o-mini
  응답: 안녕하세요, 오늘 어떻게 지내세요?


## 3-10. API 호출 비용 및 응답 시간 비교

실제 프로덕션 환경에서 고려해야 할 메트릭을 측정한다.

In [34]:
import time

def benchmark_providers(
    agent: UnifiedSkillAgent,
    skill_name: str,
    user_message: str
) -> Dict[str, Dict]:
    """
    제공자별 응답 시간을 측정한다
    
    Args:
        agent: UnifiedSkillAgent 인스턴스
        skill_name: 사용할 Skill 이름
        user_message: 사용자 메시지
        
    Returns:
        제공자별 벤치마크 결과
    """
    benchmarks = {}
    
    for provider in agent.list_available_providers():
        try:
            start_time = time.time()
            
            result = agent.execute(
                skill_name=skill_name,
                user_message=user_message,
                provider=provider
            )
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            benchmarks[provider.value] = {
                'elapsed_time': elapsed_time,
                'response_length': len(result['response']),
                'model': result['model']
            }
            
        except Exception as e:
            benchmarks[provider.value] = {'error': str(e)}
    
    return benchmarks

# 벤치마크 실행
print("\n" + "=" * 80)
print("제공자별 성능 벤치마크")
print("=" * 80)

benchmark_message = "Explain what machine learning is in simple terms.를 한국어로 번역해줘"
benchmark_results = benchmark_providers(agent, 'translator', benchmark_message)

# 결과 출력
for provider_name, metrics in benchmark_results.items():
    print(f"\n[{provider_name.upper()}]")
    if 'error' in metrics:
        print(f"  에러: {metrics['error']}")
    else:
        print(f"  모델: {metrics['model']}")
        print(f"  응답 시간: {metrics['elapsed_time']:.2f}초")
        print(f"  응답 길이: {metrics['response_length']} 문자")


제공자별 성능 벤치마크

[ANTHROPIC]
  모델: claude-sonnet-4-5
  응답 시간: 3.46초
  응답 길이: 24 문자

[OPENAI]
  모델: gpt-4o-mini
  응답 시간: 1.53초
  응답 길이: 113 문자


## 3-11. 주요 차이점 정리



| 구분 | Claude API | OpenAI API |
|------|------------|------------|
| **System Prompt 처리** | `system` 파라미터로 분리 | `messages` 배열 내 `system` role |
| **응답 구조** | `response.content[0].text` | `response.choices[0].message.content` |
| **모델 네이밍** | `claude-sonnet-4-5` | `gpt-4o-mini`,|
| **메시지 역할** | `user`, `assistant` | `system`, `user`, `assistant` |
| **라이브러리** | `anthropic` | `openai` |